In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import pickle

In [2]:
data = pd.read_csv("xAPI-Edu-Data.csv")

In [3]:
data.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   gender                    480 non-null    object
 1   NationalITy               480 non-null    object
 2   PlaceofBirth              480 non-null    object
 3   StageID                   480 non-null    object
 4   GradeID                   480 non-null    object
 5   SectionID                 480 non-null    object
 6   Topic                     480 non-null    object
 7   Semester                  480 non-null    object
 8   Relation                  480 non-null    object
 9   raisedhands               480 non-null    int64 
 10  VisITedResources          480 non-null    int64 
 11  AnnouncementsView         480 non-null    int64 
 12  Discussion                480 non-null    int64 
 13  ParentAnsweringSurvey     480 non-null    object
 14  ParentschoolSatisfaction  

In [5]:
data = data.dropna()


In [6]:
data['gender'] = data['gender'].map({'F': 0, 'M': 1})

In [7]:
X = data[['gender', 'raisedhands', 'VisITedResources', 'AnnouncementsView']]
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [8]:
model = LogisticRegression()
model.fit(X_train, y_train)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [9]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[13  1 14]
 [ 0 21  2]
 [13  7 25]]
              precision    recall  f1-score   support

           H       0.50      0.46      0.48        28
           L       0.72      0.91      0.81        23
           M       0.61      0.56      0.58        45

    accuracy                           0.61        96
   macro avg       0.61      0.64      0.62        96
weighted avg       0.61      0.61      0.61        96



In [10]:

with open("model.pkl", "wb") as file:
    pickle.dump(model, file)


In [11]:
import streamlit as st
import pickle


model = pickle.load(open("model.pkl", "rb"))


st.title("Public Speaking Confidence Predictor")


gender = st.selectbox("Gender", ["Male", "Female"])
raised_hands = st.number_input("Number of times the student raised their hand in class", value=0)
visited_resources = st.number_input("Number of times the student visited resources outside of class", value=0)
announcements_view = st.number_input("Number of times the student viewed announcements", value=0)

if st.button("Predict"):
    gender = 1 if gender.upper() == 'MALE' else 0
    raised_hands = int(raised_hands)
    visited_resources = int(visited_resources)
    announcements_view = int(announcements_view)

    inputs = [[gender, raised_hands, visited_resources, announcements_view]]
    confidence = model.predict(inputs)

    if confidence[0] == 0:
        st.write("You will be nervous during public speaking.")
    else:
        st.write("You will be confident during public speaking.")


2023-02-18 15:04:59.485 
  command:

    streamlit run D:\Anaconda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
